In [1]:
import sys
import os
import numpy as np

# Adds the parent directory to the python path
sys.path.append(os.path.abspath('../'))

from src.physics.dynamics import HSRSolver
from src.physics.simulation import MonteCarloRunner
from src.visualization.plots import plot_acm_comparison

# Setup
solver = HSRSolver()
runner = MonteCarloRunner(solver)

# Run Simulation
data = runner.run_batch_acm(n_sims=10000, n_obs=55)

# Visualize results against theoretical attractor
plot_acm_comparison(data[:, 0], data[:, 1])

Running Guided ACM:   5%|▍         | 451/10000 [01:07<23:54,  6.66it/s]  


KeyboardInterrupt: 